<a href="https://colab.research.google.com/github/Werick/Google_colab/blob/master/DTG_Summary_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [74]:
# Load the csv files
df_patient = pd.read_csv("patient_info.csv")
df_patient = df_patient.drop(columns=['patient_clinic_number','cname'])
df_patient.head()


,patient_id,DOB,age,sex,Doh,date_dtg,date_started_art,dur_prior_to_switch,height,date_height,discontinued
0,8,1955-06-15,65,0,2006-01-05,2019-02-06,2007-01-23,145,156,2020-07-08,0
1,9,1961-06-15,59,0,2006-01-27,2019-01-14,2006-07-25,150,151,2020-05-04,0
2,10,1973-06-15,47,0,2006-08-22,2018-12-19,2006-08-22,148,157,2020-05-04,0
3,12,1953-06-15,67,1,2006-03-10,2019-04-03,2006-08-29,152,176,2020-02-12,0
4,14,1975-02-23,45,0,2006-05-17,2019-02-11,2010-09-20,101,170,2020-06-02,0


In [ ]:
# get the total number of patients on this list
print("Total Population in the dataset :", df_patient.patient_id.count())

In [20]:
# Get the discription of age
df_patient.age.describe()

count    389.000000
mean      50.678663
std       11.332013
min       26.000000
25%       42.000000
50%       51.000000
75%       58.000000
max       84.000000
Name: age, dtype: float64

In [18]:
# get how many were female and how many 
group_by_gender = df_patient.patient_id.groupby(df_patient['sex']).describe()
# add precentage column to see the proportion of the population
group_by_gender['prop'] = group_by_gender.apply(lambda r:(r['count']/389)*100, axis=1)
group_by_gender[['count','prop']]

,count,prop
sex,,
0,190.0,48.843188
1,199.0,51.156812


In [21]:
# Load Longitudinal data and get the descriptive stats
df_long_data = pd.read_csv("longitudinal data.csv")
df_long_data.head()

,patient_id,visit_date,weight,height,systolic_pressure,diastolic_pressure,vl,cd_count
0,3,2018-06-11,53.0,166.0,NaN,NaN,650,NaN
1,3,2018-07-11,51.3,166.0,NaN,NaN,NaN,NaN
2,3,2018-08-07,51.0,166.0,NaN,NaN,NaN,NaN
3,3,2018-10-11,NaN,NaN,NaN,NaN,NaN,NaN
4,3,2019-01-07,55.0,166.0,NaN,NaN,1680,NaN


In [34]:
# get the list patients who have long data among the 389 patients
has_long_data = df_long_data[df_long_data['patient_id'].isin(df_patient['patient_id'])]
df_tot_patient = has_long_data.patient_id.unique()
df_long_data = has_long_data

print(len(df_tot_patient)," Patients had at least one visit")

388  Patients had at least one visit


In [36]:
# Generete a summary of visits
df_visits_summary = df_long_data.groupby(df_long_data['patient_id']).count()
df_visits_summary.head()

,visit_date,weight,height,systolic_pressure,diastolic_pressure,vl,cd_count
patient_id,,,,,,,
8,9,8,8,0,0,3,0
9,7,7,7,0,0,3,0
10,5,5,5,0,0,4,0
12,11,4,4,0,0,2,0
14,7,7,7,0,0,1,0


In [37]:
# Description of the number of visits
df_visits_summary['visit_date'].describe()

count    388.000000
mean       7.443299
std        2.162490
min        2.000000
25%        6.000000
50%        7.000000
75%        8.250000
max       17.000000
Name: visit_date, dtype: float64

In [ ]:
# Description of the number of vls
df_visits_summary['vl'].describe()

In [ ]:
# Description of the number of cd 4 counts
df_visits_summary['cd_count'].describe()

In [56]:
# Merge the two datasets
df_merged = df_visits_summary.merge(df_patient,how="inner",on="patient_id")
df_merged.head()
#Select the cols of interest
df_merged = df_merged[['patient_id','visit_date','vl','cd_count','sex']]
df_merged.head()
df_merged.groupby(df_merged['sex'])['visit_date','vl'].describe()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


visit_date                                ...   vl                    
         count      mean       std  min  25%  ...  min  25%  50%  75%  max
sex                                           ...                         
0        190.0  7.689474  2.277889  4.0  6.0  ...  1.0  2.0  3.0  3.0  5.0
1        198.0  7.207071  2.023354  2.0  6.0  ...  0.0  2.0  3.0  3.0  6.0

[2 rows x 16 columns]

In [73]:
# ART History data
df_art = pd.read_csv("art_history.csv")
df_art = df_art.drop(columns=['patient_clinic_number'])
df_art.head()

,patient_id,date_started,regimen_name,art_stop_date
0,8,2007-01-23,D4T+3TC+NVP,2013-04-18
1,8,2013-04-18,TDF+3TC+EFV,2019-02-06
2,8,2019-02-06,TDF+3TC+DTG,NaN
3,9,2006-07-25,D4T+3TC+NVP,2019-01-14
4,9,2012-06-13,3TC+NVP+TDF,NaN


In [60]:
df_art_distinct = df_art.drop_duplicates('patient_id', keep = 'first')
df_art_distinct['patient_id'].count()

389

In [68]:
df_art_summary = df_art.groupby(['regimen_name'])['patient_id'].describe()
df_art_summary

,count,mean,std,min,25%,50%,75%,max
regimen_name,,,,,,,,
3TC+NVP+TDF,100.0,649.480000,650.956475,9.0,290.75,533.0,689.75,3155.0
ABC+3TC+EFV,1.0,2870.000000,NaN,2870.0,2870.00,2870.0,2870.00,2870.0
AZT+3TC+ABC,1.0,626.000000,NaN,626.0,626.00,626.0,626.00,626.0
AZT+3TC+EFV,20.0,693.900000,710.415216,20.0,237.25,547.5,686.00,3157.0
AZT+3TC+NVP,107.0,756.485981,677.678367,14.0,365.50,586.0,924.00,3261.0
D4T+3TC+EFV,8.0,434.875000,373.685291,21.0,204.25,372.0,576.50,1187.0
D4T+3TC+NVP,88.0,592.454545,464.452933,8.0,421.25,574.5,676.25,2996.0
TDF+3TC+DTG,389.0,971.393316,985.753818,8.0,320.00,601.0,1130.00,4452.0
TDF+3TC+EFV,186.0,1242.306452,1183.642773,8.0,258.25,626.0,2744.00,4452.0
